In [1]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

True

In [2]:
# Access the environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [4]:
import os
os.environ["NEO4J_URI"]= NEO4J_URI
os.environ["NEO4J_USERNAME"]= NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]= NEO4J_PASSWORD

In [5]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [6]:
graph

In [1]:
groq_api_key = os.getenv("groq_api_key")


NameError: name 'os' is not defined

In [9]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x754f25a78150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x754f25a79850>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded online city guide 

In [12]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [13]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)


In [14]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person'),
 Node(id='Spacex', type='Company'),
 Node(id='Tesla, Inc.', type='Company'),
 Node(id='X Corp.', type='Company'),
 Node(id='Twitter', type='Company'),
 Node(id='The Boring Company', type='Company'),
 Node(id='Xai', type='Company'),
 Node(id='Neuralink', type='Company'),
 Node(id='Openai', type='Company'),
 Node(id='Forbes', type='Organization'),
 Node(id='Pretoria', type='City'),
 Node(id='Canada', type='Country'),
 Node(id='University Of Pretoria', type='University'),
 Node(id="Queen'S University", type='University'),
 Node(id='University Of Pennsylvania', type='University'),
 Node(id='California', type='State'),
 Node(id='Stanford University', type='University'),
 Node(id='Zip2', type='Company')]

In [15]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Spacex', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Tesla, Inc.', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='X Corp.', type='Company'), type='OWNER'),
 Relationship(source=Node(id='X Corp.', type='Company'), target=Node(id='Twitter', type='Company'), type='FORMER_NAME'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='The Boring Company', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Xai', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Neuralink', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person'), target=Node(id='Openai', type='Company'), type='FOUNDER'),
 R

In [17]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [18]:
graph.query(movie_query)

[]

In [19]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [20]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x754f7177fa50>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x754f25a78150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x754f25a79850>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))), qa_chain=

In [21]:
response=chain.invoke({"query":"Who is the daughter of Akash?"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Akash"})-[:PARENT_OF]->(d:Person) RETURN d.name 



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: PARENT_OF)} {position: line: 1, column: 36, offset: 35} for query: 'MATCH (p:Person {name: "Akash"})-[:PARENT_OF]->(d:Person) RETURN d.name \n'


Full Context:
[]

> Finished chain.


{'query': 'Who is the daughter of Akash?',
 'result': "I don't know the answer. \n"}

In [22]:
response=chain.invoke({"query":"tell me the genre of th movie GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[r:IN_GENRE]->(g:Genre) RETURN g.name  

Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Action'}, {'g.name': 'Thriller'}]

> Finished chain.


{'query': 'tell me the genre of th movie GoldenEye',
 'result': "I don't know the answer.  \n"}

In [23]:
response=chain.invoke({"query":"Who was the director in movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': 'Martin Scorsese  \n'}